## t5 모델 올라가는지 테스트

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer

In [6]:
model = T5ForConditionalGeneration.from_pretrained('/kaggle/input/testing-upload-t5-base-model')

In [7]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/testing-upload-t5-base-model')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
inputs = ["generate some interesting story"]
inputs = tokenizer(inputs, max_length=16, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=16, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

In [9]:
print(decoded_output)

some interesting details about the lu ft han sa rba nk


## 데이터셋 올라가는지 테스트 

In [16]:
import pandas as pd

## train, test csv 데이터셋 읽어오기 테스트
df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
df_test  = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
df_samp = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/sample_submission.csv')
df_extra = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv')
df_extra_RO = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv')

In [21]:
## faiss 깔기 위해서 캐글 다른 노트북에 올라와 있는 것을 가져와서 install 해주는 것이 필요함

!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [48]:
from faiss import write_index, read_index

## wikipedia faiss index 불러오기 테스트
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

In [30]:
## sentence transforemr 설치해주기 
## sentence-trasnformers-2.2.2 를 input에 넣어줘야 함
## llm-whls 도 같이 넣어줘야함
## 그래야 kaggle/working 에 sentencetransformer가 생김

!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing ./sentence-transformers
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126125 sha256=29c652f24ebc2fea68054e70fc5a90b615188cb74983ea33ca6ff7a2a59b39c9
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205

In [33]:
## sentencetransformers 모델 불러오기 
## (지금은 기존 kaggle 노트북에 있던 것을 가져오기, 나중에는 인코더 따로 학습시키거나 성능 좋은 모델을 가져와도 좋을 것으로 보임)

import os
from sentence_transformers import SentenceTransformer

## 기본적인 Hyperparameters 설정
SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 32
WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)

model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half() ## half를 하는 것은 embedding을 FP16으로 놓고 memory 사용량을 줄이기 위함

In [49]:
## 예시 문장 두개만 일단 임베딩 생성해보기 

import gc

prompt_embeddings = model.encode(["artificial intelligence", "data science"], batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
## 가장 관련있는 인덱스를 먼저 서치하기
## top_6의 문서 인덱스를 찾은 것

search_score, search_index = sentence_index.search(prompt_embeddings, 6)

In [51]:
## 6개의 인덱스 중 유사도 스코어가 어떻게 되는지 확인해주는 것

search_score

array([[0.72897464, 0.74723697, 0.75230414, 0.7558636 , 0.7868048 ,
        0.8105609 ],
       [0.4773128 , 0.6345405 , 0.7690647 , 0.7716476 , 0.7855485 ,
        0.7933025 ]], dtype=float32)

In [52]:
## top_6의 인덱스 (인덱스는 문서 vector의 대표 번호라고 생각하면 되는 거 같음. index.add(vector)이렇게 들어간 것으로 보임)

search_index

array([[ 368586, 4360770,  368589, 1945029, 3271967, 1086190],
       [1235496, 1235258, 2609004, 3410896, 5603365, 1235691]])

In [53]:
## 메모리 관리를 위해서 필요하지 않은 것들은 미리 지우기

import ctypes
libc = ctypes.CDLL("libc.so.6")

del sentence_index
del prompt_embeddings
_ = gc.collect()
libc.malloc_trim(0)

1

In [41]:
## parquet 파일 불러오기 (wikipedia 문서 본문을 의미하는 것 같음)
## parquet은 열기반 데이터 압축 시스템이라고 볼 수 있고, 필요한 columns을 불러오는 데에 용이하다고 볼 수 있다. 

df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                     columns=['id', 'file'])

In [57]:
print(type(zip(search_score, search_index)))

<class 'zip'>


In [58]:
import tqdm
## tqdm 오류가 나서 일단은 빼놓았음
## prompt 별로 관련있는 file이 무엇인지 concat하여 dataframe으로 반환하는 함수

wikipedia_file_data = []

for i, (scr, idx) in enumerate(zip(search_score, search_index)):
    scr_idx = idx
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

## Save memory - delete df since it is no longer necessary
del df
_ = gc.collect()
libc.malloc_trim(0)

1

In [59]:
wikipedia_file_data

,id,prompt_id,file
0,33900354,0,a.parquet
1,5841092,0,a.parquet
2,2911050,0,c.parquet
3,35458904,1,d.parquet
4,49954680,1,d.parquet
5,65583582,1,d.parquet
6,50336055,0,g.parquet
7,7827419,1,j.parquet
8,10556491,0,m.parquet
9,54415758,1,m.parquet


In [60]:
## 각 파일의 문서 내용을 다시 읽어들이는 함수라고 볼 수 있음

wiki_text_data = []

for file in wikipedia_file_data.file.unique():
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

    _df_temp = _df[_df['id'].isin(_id)].copy()
    del _df
    _ = gc.collect()
    libc.malloc_trim(0)
    wiki_text_data.append(_df_temp)
wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

In [61]:
wiki_text_data

,id,text
0,5841092,Artificial Intelligence is a scientific journa...
1,33900354,The Artificial Intelligence Center is a labora...
2,2911050,Computational Intelligence is a peer-reviewed ...
3,49954680,thumb|The institute as seen from the main walk...
4,35458904,Data science is an interdisciplinary academic ...
5,65583582,Datacommons.org is an open knowledge graph hos...
6,50336055,This glossary of artificial intelligence is a ...
7,7827419,John Yen is Professor of Data Science and Prof...
8,10556491,Machine Intelligence may refer to: * Artificia...
9,54415758,A Master of Science in Data Science is an inte...


In [65]:
## 이렇게 하면 위키 텍스트를 불러올 수 있음을 확인할 수 있음

wiki_text_data['text'].tolist()[0]

'Artificial Intelligence is a scientific journal on artificial intelligence research. It was established in 1970 and is published by Elsevier. The journal is abstracted and indexed in Scopus and Science Citation Index. The 2021 Impact Factor for this journal is 14.05 and the 5-Year Impact Factor is 11.616.Journal Citation Reports 2022, Published by Thomson Reuters ==References== == External links == * Official website Category:Artificial intelligence publications Category:Computer science journals Category:Elsevier academic journals Category:Academic journals established in 1970'

### 여기까지 공부한 것 정리 
* 모델 올리는 것은 오른편에 add data 한 다음에 내 파일을 올리면 된다. 거기서 로드해올 수 있는데, 어느 정도 크기까지가 가능할지는 좀 더 실험이 필요할 것으로 보인다.
* sentence trasnformer로 query를 임베딩 하는 방법을 터득했다. kaggle에서 제공하는 sentencetransformer를 사용하는 것도 괜찮을 것 같고, 여유가 되면 다른 모델을 가져와서 사용해도 될 것 같다.
* faiss wikipedia dataset을 이용하는 방법에 대하여 공부하였다. index를 먼저 서칭하게 된다. index는 parquet으로 되어 있는 데이터셋 중에서 가장 관련있는 번호를 뽑을 수 있게 해준다. 즉, index는 (id, 벡터) 로만 기억하고 있어서 어느 파일에 있는지 찾아갈 수 있게해준다. 이후에 parquet을 읽어서 직접적인 텍스트를 읽어올 수 있게 되면 된다. 